In [6]:
var('q t')
pari.allocatemem(9589934592)

def vector_sum(u, v):
    return dict((x, u.get(x, 0) + v.get(x, 0)) for x in set(u)|set(v))

def vector_simpl(u):
    v = {}
    S = set()
    for a in set(u):
        b = sorted(a)
        b.reverse()
        b = tuple(b)
        S.add(b)
        v = vector_sum(v, {b:u[a]})
    return {a:v[a] for a in S if v[a] != 0}

def remove_zeros(u):
    return {a:u[a] for a in set(u) if u[a] != 0}

def univ_vector_simpl1(a):
    for i in range(len(a)-1):
        if a[i]<a[i+1]:
            return {a[0:i]+(a[i]+a[i+1],)+a[i+2:len(a)]:a[i+1]-a[i], a[0:i]+(a[i+1],)+(a[i],)+a[i+2:len(a)]:1}
    return {a:1}

def univ_vector_simpl(u):
    v={}
    for a in set(u):
        w = univ_vector_simpl1(a)
        v = vector_sum(v, {b:w[b]*u[a] for b in set(w)})
    if v == u:
        return v
    return univ_vector_simpl(v)

def scalar_times_vector(u, v):
    w = {}
    for a in set(u):
        for b in set(v):
            w = vector_sum(w, {a+b:u[a]*v[b]})
    return vector_simpl(w)

def constant_times_vector(t, u):
    return {a:t*u[a] for a in set(u)}

def several_virasoro(a, u):
    return univ_vector_simpl({a+b:u[b] for b in set(u)})

def Partitions1(n):
    if n < 0:
        return [[]]
    a=list(Partitions(n))
    a.reverse()
    return a

def ordered_partitions1(n, p):
    if n < 0:
        return [[]]
    a = Partitions1(n)
    inp = ''
    w = [a[i] for i in range(len(a)) if my_len(a[i])==p]
    for i in range(len(a)):
        if my_len(a[i])==p:
            inp = inp+prt2(a[i]) + ','
    inp = inp[:-1]
    st = 'option(noredefine);'
    st = st + 'ring R = QQ, (x(2..' + str(n) + ')), (dp, C);'
    st = st + 'module M =' + inp + ';'
    st = st + 'intvec resultt1 = sort(M)[2];'
    st = st + 'intvec resultt = 0:size(resultt1);'
    st = st + 'for(int i=1; i<=size(resultt); i++){resultt[i]=resultt1[size(resultt)+1-i]-1;}'
    st = st + 'resultt;'
    st = singular.eval(st)
    b = eval(st)
    ww = [w[b[i]] for i in range(len(b))]
    return ww

def newPartitions1(n):
    if n < 0:
        return [[]]
    if n == 0:
        return [[]]
    if n == 1:
        return [[1]]
    if n == 2:
        return [[2],[1,1]]
    if n == 3:
        return [[2,1],[1,1,1],[3]]
    w = []
    for i in range(n,3,-1):
        w = w + ordered_partitions1(n, i)
    w = w + [[n-1,1], [n]]
    return w

def is_subpartition(a,b):
    if len(a)<len(b):
        return false
    for i in range(len(a)-len(b)+1):
        if a[i:i+len(b)] == b:
            return true
    return false

def missing_partition_final(a):
    if a == []:
        return false
    if is_subpartition(a,[2]) or is_subpartition(a,[3,3]) or is_subpartition(a,[4,3,1]) or is_subpartition(a,[4,4,1]) or is_subpartition(a,[6,5,3,1]):
        return true
    if len(a)>2:
        for i in range(len(a)-2):
            if a[i]-a[i+2]<=2:
                return true
    if len(a)>3:
        for i in range(len(a)-3):
            if a[i]-a[i+3]<=3 or (a[i]-a[i+3]<=4 and a[i+1]-a[i+2]<=3):
                return true
    if len(a)>4:
        for i in range(len(a)-4):
            if a[i]-a[i+1]==1 and a[i+1]-a[i+2]==2 and a[i+2]-a[i+3]==2 and a[i+3]-a[i+4]==1:
                return true
    return false

def missing_partitions_final(n):
    a = newPartitions1(n)
    pivs = [i for i in range(len(a)) if missing_partition_final(a[i])]
    return pivs

def number_of_ones(a):
    k = 0
    for i in range(len(a)):
        if a[i]==1:
            k = k+1
    return k

def Partitions1_into_parts(n):
    a = Partitions1(n)
    b = [[]]*(n+1)
    for i in range(len(a)):
        m = 2*(len(a[i])-number_of_ones(a[i]))+number_of_ones(a[i])
        b[m]= b[m]+[i]
    return b

def remove_zero_rows(A):
    for i in range(A.nrows()):
        if tuple(A[i]) == (0,)*(A.ncols()):
            A = A.delete_rows(range(i,A.nrows()))
            return A
    return A

def intersection(U, V):
    m = U.ncols()
    W = matrix(pari.mattranspose(pari.matintersect(pari(U.transpose()), pari(V.transpose()))))
    if W.nrows()==0:
        return matrix(0, m)
    return W

def newbase_subspace(n):
    m1 = {(1,1):3, (2,):-4}
    m2 = {(1,1,1):4, (2,1):-12, (3,):3}
    a = newPartitions1(n)
    b = newPartitions1(n-2)
    c = newPartitions1(n-3)
    bm1 = [several_virasoro(tuple(b[i]), m1) for i in range(len(b))]
    cm2 = [several_virasoro(tuple(c[i]), m2) for i in range(len(c))]
    A = matrix({(i,j): bm1[i].get(tuple(a[j]), 0) for j in range(len(a)) for i in range(len(b))}, sparse = False)
    B = matrix({(i,j): cm2[i].get(tuple(a[j]), 0) for j in range(len(a)) for i in range(len(c))}, sparse = False)
    A = A.stack(B)
    A = A.rref()
    A = remove_zero_rows(A)
    return A

def base_subspace(n):
    m1 = {(1,1):3, (2,):-4}
    m2 = {(3,):5, (2,1):-4}
    a = Partitions1(n)
    b = Partitions1(n-2)
    c = Partitions1(n-3)
    bm1 = [several_virasoro(tuple(b[i]), m1) for i in range(len(b))]
    cm2 = [several_virasoro(tuple(c[i]), m2) for i in range(len(c))]
    A = matrix({(i,j): bm1[i].get(tuple(a[j]), 0) for j in range(len(a)) for i in range(len(b))}, sparse = False)
    B = matrix({(i,j): cm2[i].get(tuple(a[j]), 0) for j in range(len(a)) for i in range(len(c))}, sparse = False)
    A = A.stack(B)
    A = A.rref()
    A = remove_zero_rows(A)
    return A

def parts_up_to(n):
    return [Partitions1(i) for i in range(n+1)]

def prt2(a):
    st = ""
    m = number_of_ones(a)
    for i in range(len(a)-m):
        st = st + "x(" + str(a[i]) + ")*"
    st = st + "gen(" + str(m+1) + ")"
    return st

def prt1(u):
    st = ""
    for a in set(u):
        st = st + str(u[a]) + "*" + prt2(a) + "+"
    st = st[:-1]
    return st

def prt(w):
    st = ""
    for i in range(len(w)):
        st = st + prt1(w[i]) + ","
    st = st[:-1]
    return st

def subspace_matrix(n): 
    w = []
    q = parts_up_to(n)
    for m in range(n+1):
        a = q[m]
        A = base_subspace(m)
        B = matrix(0, len(a))
        for i in range(m+1):
            b = Partitions1_into_parts(m)[i]
            B2 = matrix(len(b), len(a))
            for j in range(len(b)):
                B2[j, b[j]] = 1
            B = B.stack(B2)    
            C = intersection(B, A)
            w.extend([remove_zeros({tuple(a[b[t]]):C[j,b[t]] for t in range(len(b))}) for j in range(C.nrows())])
    ww = [u for u in w if u != {}]
    return ww

def to_partition(a):
    b = []
    max = 0
    while(a[max] == 0):
        max = max + 1
    i = len(a)-2
    while(i!=max-1):
        b = [len(a)-i]*a[i]+b
        i = i-1
    if a[len(a)-1]==3:
        b = b + [1,1]
    if a[len(a)-1]==2:
        b = b + [1]
    return b

def to_polynomial(a,n):
    b = [0]*n
    b[n-1] = number_of_ones(a)
    for i in range(len(a)):
        if a[i]!= 1:
            b[a[i]-2] = b[a[i]-2]+1
    return b        

def avoid_up_to(n):
    w = subspace_matrix(n)
    st = 'option(noredefine);'
    st = st + 'ring R = QQ, (x(2..' + str(n) + ')), (dp, C);'
    st = st + 'module M =' + prt(w) + ';'
    st = st + 'module G = std(M);'
    st = st + 'string result = "";'
    st = st + 'for(int i = 1; i<=size(G); i++){result = result + "[" + string(leadexp(G[i])) + "],";}'
    st = st + 'result = result[1..size(result)-1];'
    st = st + 'result = "[" + result + "]";'
    st = st + 'result;'
    st = singular.eval(st)
    avoid = eval(st)
    for i in range(len(avoid)):
        avoid[i][n-1] = avoid[i][n-1]-1
    return avoid

def avoid_up_to_v0(n):
    w = subspace_matrix(n)
    st = 'option(noredefine);'
    st = st + 'ring R = QQ, (x(' + str(n) + '..2)), (dp, C);' 
    st = st + 'module M =' + prt(w) + ';'
    st = st + 'module G = std(M);'
    st = st + 'string result = "";'
    st = st + 'for(int i = 1; i<=size(G); i++){result = result + "[" + string(leadexp(G[i])) + "],";}'
    st = st + 'result = result[1..size(result)-1];'
    st = st + 'result = "[" + result + "]";'
    st = st + 'result;'
    st = singular.eval(st)
    b = eval(st)
    avoid = [[1,1,1]]
    for i in range(len(b)):
        avoid = avoid + [to_partition(b[i])]
    return avoid

def p_divides_q(p,q,n):
    for i in range(n):
        if p[i]>q[i]:
            return false
    return true

def a_contains_any_of(a, avoid, n):
    p = to_polynomial(a, n)
    if p[n-1]>2:
        return true
    for i in range(len(avoid)):
        if p[n-1] == avoid[i][n-1] and p_divides_q(avoid[i][:-1], p[:-1], n-1):
            return true
    return false

def base_up_to(n):
    base = [[[]], [[1]], [[1,1]]]
    avoid = avoid_up_to(n)
    q = parts_up_to(n)
    for i in range(3, n+1):
        add = []
        a = q[i]
        for j in range(len(a)):
            if not a_contains_any_of(a[j], avoid, n):
                add = add + [a[j]]
        base = base + [add]
    return base

def base_maybe_up_to(n):
    base = []
    for i in range(n+1):
        a = Partitions1(i)
        add = [a[j] for j in range(len(a)) if not missing_partition_final(a[j])]
        base = base + [add]
    return base

def my_len(a):
    return 2*(len(a)-number_of_ones(a))+number_of_ones(a)

def series_t_q(w):
    series = 0
    for i in range(len(w)):
        series_t = 0
        for j in range(len(w[i])):
            series_t = series_t + t^my_len(w[i][j])
        series = series + series_t*q^i
    return series

def A_1_1(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if number_of_ones(w[i][j])==2:
                a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_1(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if number_of_ones(w[i][j])==1:
                a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_g1(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if number_of_ones(w[i][j])==0:
                a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_3(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if len(w[i][j]) > 0:
                if w[i][j][len(w[i][j])-1]==3:
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_4(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if len(w[i][j]) > 0:
                if w[i][j][len(w[i][j])-1]==4:
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_g4(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if number_of_ones(w[i][j])==0:
                if len(w[i][j])==0:
                    a = a + [w[i][j]]
                else:
                    if w[i][j][len(w[i][j])-1]>4:
                        a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_g3(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if number_of_ones(w[i][j])==0:
                if len(w[i][j])==0:
                    a = a + [w[i][j]]
                else:
                    if w[i][j][len(w[i][j])-1]>3:
                        a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_4_3(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if len(w[i][j]) > 1:
                if (w[i][j][len(w[i][j])-1]==3 and w[i][j][len(w[i][j])-2]==4):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_g4_3(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if w[i][j] == [3]:
                a = a + [[3]]
            if len(w[i][j]) > 1:
                if (w[i][j][len(w[i][j])-1]==3 and w[i][j][len(w[i][j])-2]>4):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_5_3(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if len(w[i][j]) > 1:
                if (w[i][j][len(w[i][j])-1]==3 and w[i][j][len(w[i][j])-2]==5):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_g5_3(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if w[i][j] == [3]:
                a = a + [[3]]
            if len(w[i][j]) > 1:
                if (w[i][j][len(w[i][j])-1]==3 and w[i][j][len(w[i][j])-2]>5):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_4_4(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if len(w[i][j]) > 1:
                if (w[i][j][len(w[i][j])-1]==4 and w[i][j][len(w[i][j])-2]==4):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_5_4(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if len(w[i][j]) > 1:
                if (w[i][j][len(w[i][j])-1]==4 and w[i][j][len(w[i][j])-2]==5):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_g5_4(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if w[i][j] == [4]:
                a = a + [[4]]
            if len(w[i][j]) > 1:
                if (w[i][j][len(w[i][j])-1]==4 and w[i][j][len(w[i][j])-2]>5):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_g4_4(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if w[i][j] == [4]:
                a = a + [[4]]
            if len(w[i][j]) > 1:
                if (w[i][j][len(w[i][j])-1]==4 and w[i][j][len(w[i][j])-2]>4):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_6_5_3(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if len(w[i][j]) > 2:
                if (w[i][j][len(w[i][j])-1]==3 and w[i][j][len(w[i][j])-2]==5 and w[i][j][len(w[i][j])-3]==6):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_6_4_3(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if len(w[i][j]) > 2:
                if (w[i][j][len(w[i][j])-1]==3 and w[i][j][len(w[i][j])-2]==4 and w[i][j][len(w[i][j])-3]==6):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_7_4_4(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if len(w[i][j]) > 2:
                if (w[i][j][len(w[i][j])-1]==4 and w[i][j][len(w[i][j])-2]==4 and w[i][j][len(w[i][j])-3]==7):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_g6_5_3(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if w[i][j] == [5,3]:
                a = a + [[5,3]]
            if len(w[i][j]) > 2:
                if (w[i][j][len(w[i][j])-1]==3 and w[i][j][len(w[i][j])-2]==5 and w[i][j][len(w[i][j])-3]>6):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_g7_4_4(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if w[i][j] == [4,4]:
                a = a + [[4,4]]
            if len(w[i][j]) > 2:
                if (w[i][j][len(w[i][j])-1]==4 and w[i][j][len(w[i][j])-2]==4 and w[i][j][len(w[i][j])-3]>7):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_g6_4_3(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if w[i][j] == [4,3]:
                a = a + [[4,3]]
            if len(w[i][j]) > 2:
                if (w[i][j][len(w[i][j])-1]==3 and w[i][j][len(w[i][j])-2]==4 and w[i][j][len(w[i][j])-3]>6):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_4_4or6_5_3(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if len(w[i][j]) > 1:
                if (w[i][j][len(w[i][j])-1]==4 and w[i][j][len(w[i][j])-2]==4):
                    a = a + [w[i][j]]
            if len(w[i][j]) > 2:
                if (w[i][j][len(w[i][j])-1]==3 and w[i][j][len(w[i][j])-2]==5 and w[i][j][len(w[i][j])-3]==6):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_3_1(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if len(w[i][j]) > 1:
                if (w[i][j][len(w[i][j])-1]==1 and w[i][j][len(w[i][j])-2]==3):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_5_3_1(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if len(w[i][j]) > 2:
                if (w[i][j][len(w[i][j])-1]==1 and w[i][j][len(w[i][j])-2]==3 and w[i][j][len(w[i][j])-3]==5):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_g5_3_1(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if w[i][j] == [3,1]:
                a = a + [[3,1]]
            if len(w[i][j]) > 2:
                if (w[i][j][len(w[i][j])-1]==1 and w[i][j][len(w[i][j])-2]==3 and w[i][j][len(w[i][j])-3]>5):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_g4_1(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if w[i][j] == [1]:
                a = a + [[1]]
            if len(w[i][j]) > 1:
                if (w[i][j][len(w[i][j])-1]==1 and w[i][j][len(w[i][j])-2]>4):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_5_4_1(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if len(w[i][j]) > 2:
                if (w[i][j][len(w[i][j])-1]==1 and w[i][j][len(w[i][j])-2]==4 and w[i][j][len(w[i][j])-3]==5):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_g5_4_1(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if w[i][j] == [4,1]:
                a = a + [[4,1]]
            if len(w[i][j]) > 2:
                if (w[i][j][len(w[i][j])-1]==1 and w[i][j][len(w[i][j])-2]==4 and w[i][j][len(w[i][j])-3]>5):
                    a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_4_1_1(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if number_of_ones(w[i][j])==2:
                if len(w[i][j])>2:
                    if w[i][j][len(w[i][j])-3]==4:
                        a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def A_g4_1_1(w):
    ww = []
    for i in range(len(w)):
        a = []
        for j in range(len(w[i])):
            if number_of_ones(w[i][j])==2:
                if w[i][j]==[1,1]:
                    a = a + [[1,1]]
                if len(w[i][j])>2:
                    if w[i][j][len(w[i][j])-3]>4:
                        a = a + [w[i][j]]
        ww = ww + [a]
    return ww

def check_leading_monomials(n):
    piv = list(newbase_subspace(n).pivots())
    return piv == missing_partitions_final(n)

def check_leading_monomials_up_to(n):
    for i in range(n+1):
        print([i,check_leading_monomials(i)])
        
def show_pivots(n):
    piv = newbase_subspace(n).pivots()
    a = newPartitions1(n)
    return [a[i] for i in piv]

def show_pivots_up_to(n):
    for i in range(n+1):
        print((i,show_pivots(i)))

PARI stack size set to 9589934592 bytes, maximum size set to 9589936128


In [7]:
show_pivots_up_to(15)

(0, [])
(1, [])
(2, [[2]])
(3, [[2, 1], [1, 1, 1]])
(4, [[2, 2], [2, 1, 1], [1, 1, 1, 1]])
(5, [[2, 2, 1], [2, 1, 1, 1], [1, 1, 1, 1, 1], [3, 2], [3, 1, 1]])
(6, [[2, 2, 2], [2, 2, 1, 1], [2, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [3, 2, 1], [3, 1, 1, 1], [3, 3], [4, 2]])
(7, [[2, 2, 2, 1], [2, 2, 1, 1, 1], [2, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1], [3, 2, 2], [3, 2, 1, 1], [3, 1, 1, 1, 1], [3, 3, 1], [4, 2, 1], [4, 1, 1, 1], [5, 2]])
(8, [[2, 2, 2, 2], [2, 2, 2, 1, 1], [2, 2, 1, 1, 1, 1], [2, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [3, 2, 2, 1], [3, 2, 1, 1, 1], [3, 1, 1, 1, 1, 1], [3, 3, 2], [4, 2, 2], [3, 3, 1, 1], [4, 2, 1, 1], [4, 1, 1, 1, 1], [4, 3, 1], [5, 2, 1], [5, 1, 1, 1], [6, 2]])
(9, [[2, 2, 2, 2, 1], [2, 2, 2, 1, 1, 1], [2, 2, 1, 1, 1, 1, 1], [2, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1], [3, 2, 2, 2], [3, 2, 2, 1, 1], [3, 2, 1, 1, 1, 1], [3, 1, 1, 1, 1, 1, 1], [3, 3, 2, 1], [4, 2, 2, 1], [3, 3, 1, 1, 1], [4, 2, 1, 1, 1], [4, 1, 1, 1, 1, 1], [3, 3, 3], [4, 3, 2], [

In [8]:
%time check_leading_monomials_up_to(19)

[0, True]
[1, True]
[2, True]
[3, True]
[4, True]
[5, True]
[6, True]
[7, True]
[8, True]
[9, True]
[10, True]
[11, True]
[12, True]
[13, True]
[14, True]
[15, True]
[16, True]
[17, True]
[18, True]
[19, True]
CPU times: user 6.49 s, sys: 421 ms, total: 6.91 s
Wall time: 7.18 s


In [9]:
base_up_to(15)

[[[]],
 [[1]],
 [[1, 1]],
 [[3]],
 [[3, 1], [4]],
 [[4, 1], [5]],
 [[4, 1, 1], [5, 1], [6]],
 [[4, 3], [5, 1, 1], [6, 1], [7]],
 [[4, 4], [5, 3], [6, 1, 1], [7, 1], [8]],
 [[5, 3, 1], [5, 4], [6, 3], [7, 1, 1], [8, 1], [9]],
 [[5, 4, 1], [5, 5], [6, 3, 1], [6, 4], [7, 3], [8, 1, 1], [9, 1], [10]],
 [[5, 5, 1],
  [6, 4, 1],
  [6, 5],
  [7, 3, 1],
  [7, 4],
  [8, 3],
  [9, 1, 1],
  [10, 1],
  [11]],
 [[5, 5, 1, 1],
  [6, 4, 1, 1],
  [6, 5, 1],
  [6, 6],
  [7, 4, 1],
  [7, 5],
  [8, 3, 1],
  [8, 4],
  [9, 3],
  [10, 1, 1],
  [11, 1],
  [12]],
 [[6, 4, 3],
  [6, 5, 1, 1],
  [6, 6, 1],
  [7, 4, 1, 1],
  [7, 5, 1],
  [7, 6],
  [8, 4, 1],
  [8, 5],
  [9, 3, 1],
  [9, 4],
  [10, 3],
  [11, 1, 1],
  [12, 1],
  [13]],
 [[6, 5, 3],
  [6, 6, 1, 1],
  [7, 4, 3],
  [7, 5, 1, 1],
  [7, 6, 1],
  [7, 7],
  [8, 4, 1, 1],
  [8, 5, 1],
  [8, 6],
  [9, 4, 1],
  [9, 5],
  [10, 3, 1],
  [10, 4],
  [11, 3],
  [12, 1, 1],
  [13, 1],
  [14]],
 [[6, 6, 3],
  [7, 4, 4],
  [7, 5, 3],
  [7, 6, 1, 1],
  [7, 7, 1],
 

In [10]:
base_maybe_up_to(30)

[[[]],
 [[1]],
 [[1, 1]],
 [[3]],
 [[3, 1], [4]],
 [[4, 1], [5]],
 [[4, 1, 1], [5, 1], [6]],
 [[4, 3], [5, 1, 1], [6, 1], [7]],
 [[4, 4], [5, 3], [6, 1, 1], [7, 1], [8]],
 [[5, 3, 1], [5, 4], [6, 3], [7, 1, 1], [8, 1], [9]],
 [[5, 4, 1], [5, 5], [6, 3, 1], [6, 4], [7, 3], [8, 1, 1], [9, 1], [10]],
 [[5, 5, 1],
  [6, 4, 1],
  [6, 5],
  [7, 3, 1],
  [7, 4],
  [8, 3],
  [9, 1, 1],
  [10, 1],
  [11]],
 [[5, 5, 1, 1],
  [6, 4, 1, 1],
  [6, 5, 1],
  [6, 6],
  [7, 4, 1],
  [7, 5],
  [8, 3, 1],
  [8, 4],
  [9, 3],
  [10, 1, 1],
  [11, 1],
  [12]],
 [[6, 4, 3],
  [6, 5, 1, 1],
  [6, 6, 1],
  [7, 4, 1, 1],
  [7, 5, 1],
  [7, 6],
  [8, 4, 1],
  [8, 5],
  [9, 3, 1],
  [9, 4],
  [10, 3],
  [11, 1, 1],
  [12, 1],
  [13]],
 [[6, 5, 3],
  [6, 6, 1, 1],
  [7, 4, 3],
  [7, 5, 1, 1],
  [7, 6, 1],
  [7, 7],
  [8, 4, 1, 1],
  [8, 5, 1],
  [8, 6],
  [9, 4, 1],
  [9, 5],
  [10, 3, 1],
  [10, 4],
  [11, 3],
  [12, 1, 1],
  [13, 1],
  [14]],
 [[6, 6, 3],
  [7, 4, 4],
  [7, 5, 3],
  [7, 6, 1, 1],
  [7, 7, 1],
 